In [1]:
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
from PIL import Image
from PIL import ImageFilter

In [2]:
def preprocessing(result) :
    img_gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY) # แปลงภาพ result ให้เป็นภาพ Gray
    img = cv2.cvtColor(result, cv2.COLOR_BGR2RGB) # แปลงภาพ result ให้เป็นภาพ RGB
    return (img, img_gray) # return ภาพ img และ img_gray

In [3]:
def stitch_and_crop(ref_img, query_img, H) : # Warp and stitch
    rows, cols = ref_img.shape[:2] # เตรียมข้อมูลเป็น rows กับ cols
    warped_img = cv2.warpPerspective(query_img, H, (cols, rows)) #จากนั้นทำการ warp ภาพ โดยใช้ฟังก์ชั่น cv2.warpPerspective โดยใช้ภาพ query_img
    enlarge_ref_img = np.zeros((1*rows, 1*cols, 3), dtype=np.uint8) # จากนั้นส่งอาเรย์ 0 ไปยัง rows และ cols
    enlarge_ref_img[:rows, :cols] = ref_img  #จากนั้น ให้ enlarge_ref_img[:rows, :cols] เท่ากับ ref_img
    warped_gray = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY) # แปลงสีภาพ warped_img ให้เป็น gray 
    warped_mask = cv2.threshold(warped_gray, 0, 255, cv2.THRESH_BINARY)[1] # ทำการ mask ที่ได้จากการแปลง เป็น Gray โดยใช้ฟังก์ชั่น cv2.threshold
    segmented_ref_img = cv2.add(warped_img, enlarge_ref_img, mask=np.bitwise_not(warped_mask)) # จากนั้นก็บวกภาพ warped_img และ enlarge_ref_img ไม่ใช่ที่ warped_mask
    result = cv2.add(segmented_ref_img, warped_img) # บอกภาพ Segmented_ref_img กับ warped_img

    grayscale = cv2.cvtColor(result, cv2.COLOR_RGB2GRAY) # จากนั้นแปลงภาพ result ให้เป็นภาพ gray
    mask = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)[1] # mask ภาพ grayscale โดยใช่ฟังก์ชั่น cv2.threshold
    
    contour, hier = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # หา contour ใน mask

   
    max_area_contour = max(contour, key=cv2.contourArea) # ให้ max_area_contour เท่ากับพื้นที่สูงสุดของ comtour 

   
    (x, y, w, h) = cv2.boundingRect(max_area_contour) # สร้างขอบ ใน max_area_contour โดยใช้ฟังก์ชั่น cv2.boundingRect

    return result[:h, :w]

In [4]:
tmep_list = list() #สร้างตัวแปร lsit
for i in range(1,5) :  # วนค่า i ใน range 1,5
    img = cv2.imread('./images/final_exam/Templates/Template-{}.png'.format(i)) #อ่านภาพแล้วที่มีตามค่า i 
    tmep_list.append(img) #เก็บภาพไว้ในตัวแปร temp_list
def image_stitching(tmep_list) : 
  ref_img = tmep_list[0]  # ให้ ref_img มีค่าเท่ากับภาพแรกของ temp_list
  for i in range(4) :  # วนค่า i 4 ค่า
    ref_img = ref_img  # ให้ ref_img เท่ากับ ref_img เนื่องจาก ref_img ที่ได้จากการวนครั้งแรกหรือครั้งถัดๆไป เป็น input ในการต่อภาพถัดๆไป
    if i < 3 :    # ถ้า น้อยกว่า 6
      query_img= tmep_list[i+1]   #ให้ query_img เท่ากับ img_list[i+1] เพื่อที่จะถึงภาพถัดไปมาต่อกับ ref_img
    ref_gray = cv2.cvtColor(ref_img,cv2.COLOR_BGR2GRAY)   # แปลงภาพ ref_img ให้เป็น ภาพ gray รอจับคู่ match  เนื่องจากแปลงภาพเป็น gray ตรวจจับจุด match ได้ง่ายว่าภาพสี
    query_gray = cv2.cvtColor(query_img,cv2.COLOR_BGR2GRAY)  # แปลงภาพ query_img ให้ภาพ gray รอจับคู่ match เนื่องจากแปลงภาพเป็น gray ตรวจจับจุด match ได้ง่ายว่าภาพสี
    sift = cv2.SIFT_create()     # สร้าง sift ขึ้นมา
    ref_kpts, ref_desc = sift.detectAndCompute(ref_gray, None)  #สร้าง keypoint และ descriptor เพื่อตรวจจับภาพ ref_gray
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None) #สร้าง keypoint และ descriptor เพื่อตรวจจับภาพ query_gray

    bf = cv2.BFMatcher()    # สร้างตัวแปร bf รอการ match ข้อมูล    
    matches = bf.knnMatch(ref_desc,query_desc, k =2) # match ข้อมูลระหว่าง ref_desc และ query_desc
    good_matches = list()         #สร้างตัวแปร list รอเก็บค่า
    good_matches_list = list()    #สร่างตัวแปร list รอเก็บค่า
    for m, n in matches :         # วนค่า m , n ใน matches
      if m.distance < 0.7*n.distance :   # ถ้า m.distance น้อยกว่า 0.7*n.distance เพือหาจุดเด่นของภาพทั้ง 2 ออกมา
          good_matches.append(m)         # เก็บค่าของ m ไว้ใน good_matches
          good_matches_list.append([m])   # เก็บค่าของ n ไว้ใน good_matches_list

    ratio_matched_img = cv2.drawMatchesKnn(ref_img, ref_kpts,query_img,query_kpts,good_matches_list , None, flags=2) # วาดคู่ match ที่เหมือนกันระหว่างภาพ ref_img และ query_img 

    MIN_MATCH_NUMBER = 10  # กำหนดคู่ match ที่จะทำไปใช้ อย่างน้อย 10 ตัว
    if len(good_matches) > MIN_MATCH_NUMBER : # ถ้า good_matches มากกว่า MIN_MATCH_NUMBER 
      print('Enough matched feature')
      tp = np.float32([ ref_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)   # หาตำแหน่งภาพของ ref_kpts  แล้วทำการ reshape ให้เป็น เมทริกซ์ 1 แถว 2 หลัก เพื่อรอการ wrap
      fp = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2) # หาตำแหน่งภาพของ query_kpts แล้วทำการ reshape ให้เป็น เมททริกซ์ 1 แถว 2 หลัก เพื่อรอการ wrap
      
      H, inlier_masks = cv2.findHomography(fp, tp, cv2.RANSAC, 10.0) # กรองจุด match ที่ match กันผิดพลาดซึ่งจะรับได้อยู๋ที่ 10 px
      ransac_img = cv2.drawMatchesKnn(ref_img,ref_kpts,query_img,query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks) # วาดคู่ match ที่ได้จากการกรองจุด match ที่ผิดพลาดออกไป ระหว่าง ref_img และ query_img
    result = stitch_and_crop(ref_img,query_img, H) # เรียกใช้ฟังก์ชั่น stitch_and_crop แล้วส่งค่าตัวแปร ref_img query_img H เพื่อทำการ crop และ wrap ภาพ
    plt.figure(figsize=(20,20))
    ref_img = result  # ให้ ref_img เท่ากับ result เพื่อเป็น input ให้กับการต่อภาพถัดไป
  return result
result = image_stitching(tmep_list) #ส่ง temp_list เข้าไปยังฟังก์ชั่น image_stitiching

Enough matched feature
Enough matched feature
Enough matched feature
Enough matched feature


<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

<Figure size 1440x1440 with 0 Axes>

In [5]:
result =cv2.bilateralFilter(result, 12,55,100) # ใช้ฟังก์ชั่น cv2.bilateralFilter เพื่อทำให้ภาพมีความ คมชัดมากขึ้น 
template_img, template_gray = preprocessing(result) # จากนั้นเรียกใช่ฟังก์ชั่น perprocessing แล้วส่งภาพ result ไป

In [6]:
sift = cv2.SIFT_create() #สร้าง sift ขึ้นมา
bf = cv2.BFMatcher() #สร้างตัวแปร bf เพื่อรอ match ข้อมูล

In [7]:
vid = cv2.VideoCapture('./videos/final_exam/Dataset-2/right_output.avi') #เปิดวิดีโอ
def feature_object_detection(template_img, template_gray, frame,frame_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None) # detect หาจุดระหว่าง template_kpts กับ template_desc ในภาพ template_gray
    query_kpts, query_desc = sift.detectAndCompute(frame_gray, None) # detect หาจุดระหว่าง query_kpts กับ query_desc ใน frame_gray
    matches = bf.knnMatch(template_desc, query_desc, k=2)  # ทำการ match กันระหว่าง template_desc กับ query_desc
    good_matches = list() 
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.8*n.distance : 
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number : # ถ้า good_matches มากกว่า min_match_number
        # วน ค่า m ใน good_matches จากนั้นก็มาเรียกหา queryIdx และ template_kpts จากนั้นก็เอา keypoint ออกมา และ รีshape ที่มี  1 แถว 2 หลัก 
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2) # -1 คือ จะมี shape ยังไงก็ได้
        # วน ค่า m ใน good_matches จากนั้นก็มาเรียกหา trainIdx และ query_kpts จากนั้นก็เอา keypoint ออกมา และ รีshape ที่มี  1 แถว 2 หลัก เพื่อหาจุดที่ match กันระหว่าง src_pts และ dst_pts
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
        #จากนั้นจับคู่จุดจุดหนึ่งไปหาอีกจุดหนึ่งที่สอดคล้องกันระหว่าง src_pts และ dst_pts โดยใช้วิธีการของ RANSAC และมีค่า error ได้ที่ 5 พิเซล
        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,3.5) #5
        # get the bounding box around template image
        h, w = template_img.shape[:2] # เรียก cols และ rows ของภาพ template_img ให้เอากับ h,w 

        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2) # ตีกรอบ
        # เรียกใช้ฟังก์ชั่น cv2.perspectiveTransform (ซึ่งจะรับข้อมูลที่เป็น point) ในการ transform template_box ด้วย martix H
        transformed_box = cv2.perspectiveTransform(template_box, H) 
        
        detected_img = cv2.polylines(frame, [np.int32(transformed_box)], True, (0,255,0), 3, cv2.LINE_AA) # transformed_box วาดเข้าไปใน frame 
        #ทำการ match กันระว่างภาพ template_kpts และ query_kpts (flages คือ การวาด) จะเอาเฉพาะที่อยู่ใน inlier_masks
        drawmatch_img = cv2.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)
        
        cv2.imshow('frame', detected_img)
        cv2.imshow('frame1',drawmatch_img)
        return detected_img, drawmatch_img 
    else :
        return 
    

In [8]:
while vid.isOpened() :
    ret, frame = vid.read() #อ่าน ret และ frame จาก วิดีโอ
    ret1, frame1 = vid.read()#อ่าน ret และ frame จาก วิดีโอ
    frame = cv2.bilateralFilter(frame, 3, 75, 100)
    freme1=cv2 .morphologyEx ( frame1 ,  cv2 . MORPH_CLOSE , (5,5)) 
    freme1=cv2 .morphologyEx ( frame1 ,  cv2 . MORPH_OPEN  , (5,5))
    frame1 = cv2.bilateralFilter(frame1, 3, 55, 100)
    if ret :
        
        hsv_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2HSV) 
        upper = np.array([30,255,255]) 
        lower = np.array([20,120,100]) 
        yellow_frame = cv2.inRange(hsv_img,lower,upper)

        upper = np.array([10,255,100]) 
        lower = np.array([0,100,50]) 
        red_frame = cv2.inRange(hsv_img,lower,upper)

        test = np.ones_like(red_frame,np.uint8)*255
        sum_color = cv2.bitwise_or(yellow_frame,red_frame,mask= test)

        contours,hierarchys = cv2.findContours(sum_color ,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)                                                                                            
        mask= np.zeros(sum_color.shape[:2],np.uint8) 
        for X in range(len(contours)):     
            if cv2.contourArea(contours[X])>600 :  
                cv2.drawContours(mask,contours,X,(255,255,255),35)
                cv2.drawContours(mask,contours,X,(255,255,255),-1)

        mask_1 = cv2.bitwise_and(frame,frame,mask=mask)
        
        mask_1_gray = cv2.cvtColor( mask_1, cv2.COLOR_BGR2GRAY)
        mask_1_gray=mask_1_gray+mask_1_gray
        detected, drawmatch =  feature_object_detection(template_img, template_gray,frame,mask_1_gray,3) #3
        if cv2.waitKey(1) & 0xFF == ord('q') : 
            break
    else :
        break
cv2.destroyAllWindows()